# Snapshot ImageNet volumes

In [33]:
#import aws
import os
import sys
import time
#import tensorflow as tf
import boto3

sys.path.insert(0, os.environ['HOME']+'/git0/cluster')
import util as u

import importlib; importlib.reload(u)
import boto3
from collections import OrderedDict, defaultdict

os.environ['AWS_DEFAULT_REGION']='us-east-1'
client = u.create_ec2_client()
ec2 = u.create_ec2_resource()

## List volumes

In [2]:
volumes = list(ec2.volumes.all())
for v in volumes:
  if v.attachments:
    machine = u.get_name(ec2.Instance(v.attachments[0]['InstanceId']))
  else:
    machine = '<unattached>'
  print(v.id, u.get_name(v), machine)

vol-0a1265aa66e50ec14 imagenet_blank <unattached>
vol-082fde9b0b4d2c9b1 imagenet_00 <unattached>
vol-009d7afd9b87f8a61 imagenet_15 <unattached>
vol-0348dc4edaf0f1be2 imagenet_10 <unattached>
vol-057e8c8c5661da393 imagenet_01 <unattached>
vol-0468d7685e6aa3b4f imagenet_08 <unattached>
vol-0be0410ea1d6f3d37 imagenet_04 <unattached>
vol-08a31ce44d1c53565 imagenet_06 <unattached>
vol-07c52c70b8066a682 imagenet_03 <unattached>
vol-0b3d3969f8469273e imagenet_05 <unattached>
vol-09bf2755dbe29762a imagenet_14 <unattached>
vol-0246fea91e3aefe45 imagenet_07 <unattached>
vol-0737306dae8d9f600 imagenet_09 <unattached>
vol-07898d991ad5b2ec4 imagenet_13 <unattached>
vol-0fcc55f82abd8187c imagenet_12 <unattached>
vol-0de15547a309a9996 imagenet_11 <unattached>
vol-03b0ac2225356b5ad imagenet_02 <unattached>
vol-0e45af450f03103e6 imagenet_24 <unattached>
vol-0a160f8959801bb8e imagenet_17 <unattached>
vol-0e0655e5c98930d08 imagenet_26 <unattached>
vol-0447b457352cc35f4 imagenet_19 <unattached>
vol-087efb

In [4]:
def get_volume_dict():
  """Returns dictionary of named volumes"""
  volumes = list(ec2.volumes.all())
  d = {}
  for v in volumes:
    name = u.get_name(v)
    if name != u.EMPTY_NAME:
      d[name] = v
  return d

In [5]:
volume_dict = get_volume_dict()

In [6]:
volume_dict['imagenet_blank']

ec2.Volume(id='vol-0a1265aa66e50ec14')

## Get volume name

In [94]:
v.availability_zone

'us-east-1a'

In [ ]:
v.de

In [92]:
ec2.volumes.filter?

In [91]:
volumes = list(ec2.volumes.all())
for v in volumes:
  if v.attachments:
    machine = u.get_name(ec2.Instance(v.attachments[0]['InstanceId']))
  else:
    machine = '<unattached>'
  print(v.id, u.get_name(v), machine)

vol-0a1265aa66e50ec14 imagenet_blank <unattached>
vol-01618de9f651d2d97 imagenet_00 <unattached>
vol-0a81561fcdad2b40d imagenet_01 <unattached>
vol-02c96004b2457e1f0 imagenet_04 <unattached>
vol-035672ec5f764f1e0 imagenet_03 <unattached>
vol-06a91fb406ac2fa80 imagenet_07 <unattached>
vol-0ad5ecc3a0fb9e47f imagenet_05 <unattached>
vol-02686c94328ac66eb imagenet_02 <unattached>
vol-087b30e9f1ce205c1 imagenet_06 <unattached>
vol-0d723324500b507d7 imagenet_08 <unattached>
vol-0b626f58e9bcaa33b imagenet_10 <unattached>
vol-090c146550efe47d7 imagenet_15 <unattached>
vol-011a3b23e48f59a13 imagenet_14 <unattached>
vol-0499caaf9a0add8bc imagenet_09 <unattached>
vol-05fbc56bb30641495 imagenet_11 <unattached>
vol-0d56d4cbcb7b3fb48 imagenet_13 <unattached>
vol-083a5365d789b516b imagenet_12 <unattached>
vol-08fec5960e48515b9  0.tb.monitoring
vol-0f0e841d0cc657002 a.etcd-main.df86c4e8-pachydermcluster.kubernetes.com master-us-east-1a.masters.df86c4e8-pachydermcluster.kubernetes.com
vol-06fb03280cf25

In [6]:
def create_blank_volume(name, zone, size, iops, voltype='io1'):
  tag_specs = [{
    'ResourceType': 'volume',
    'Tags': [{
        'Key': 'Name',
        'Value': name
    }]
  }]
  volume = ec2.create_volume(Size=size, VolumeType=voltype,
                             TagSpecifications=tag_specs,
                             AvailabilityZone=zone,
                             Iops=iops)
  return volume

In [99]:
vol.volume_type

'io1'

In [22]:
# imagenet is 200GB, so make 300GB just in case
vol = create_blank_volume("imagenet_blank", "us-east-1c", size=300, iops=10*1000)

In [14]:
def lookup_volume(name):
  vols = []
  for v in ec2.volumes.all():
    if u.get_name(v) == name:
      vols.append(v)
  assert len(vols)>0, f"volume {name} not found"
  assert len(vols)<2, f"multiple volumes with name={name}"
  return vols[0]

In [33]:
ATTACH_WAIT_INTERVAL_SEC=5
def attach_vol(vol, instance, device='/dev/xvdf'):
  while True:
    try:
      response = vol.attach_to_instance(InstanceId=instance.id, Device=device)
      print(f'Attaching {u.get_name(vol)} to {u.get_name(instance)}: response={response.get("State", "none")}')
    except Exception as e:
      print(f'Error attaching volume: ({e}). Retrying in {ATTACH_WAIT_INTERVAL_SEC}', e)
      time.sleep(ATTACH_WAIT_INTERVAL_SEC)
      continue
    else:
      print('Attachment successful')
      break

In [34]:
vol = u.lookup_volume('imagenet_blank')
instance = u.get_instance('va')

In [36]:
u.get_name(instance)

'0.worker.eight-machines-va02'

In [35]:
attach_vol(vol, instance)

Attaching imagenet_blank to 0.worker.eight-machines-va02: response=attaching
Attachment successful


## Mount volume

In [61]:
import aws_backend
run = aws_backend.make_run('new-test-run')
job = run.make_job('worker', skip_existing_job_validation=True, instance_type='asdf',
                  availability_zone='asdf')

DataNotFoundError: Unable to load data for: endpoints

In [68]:
/dev/nvme0n1p1                                     8376300    1130668          7245632  14% /


In [41]:
job.wait_until_ready()

2018-07-30 16:42:48.405044 0.worker.eight-machines-va02: Running initialize
ssh_to_host ubuntu@34.207.107.99
2018-07-30 16:42:50.742881 0.worker.eight-machines-va02: tmux> mkdir -p /tmp/tmux
2018-07-30 16:42:52.399528 0.worker.eight-machines-va02: Mounting EFS
2018-07-30 16:42:53.460376 0.worker.eight-machines-va02: tmux> sudo mkdir -p /efs
2018-07-30 16:42:55.073735 0.worker.eight-machines-va02: tmux> sudo chmod 777 /efs
2018-07-30 16:42:56.702661 0.worker.eight-machines-va02: tmux> sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 fs-de3d0697.efs.us-east-1.amazonaws.com:/ /efs
2018-07-30 16:42:58.334179 0.worker.eight-machines-va02: Warning: command sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 fs-de3d0697.efs.us-east-1.amazonaws.com:/ /efs returned status 32
2018-07-30 16:42:58.334393 0.worker.eight-machines-va02: Checking for /tmp/is_initialized file
2018-07-30 16:42:59.196061 0.worker.eight-machines-va02: 

In [45]:
#Follow recipe on https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ebs-using-volumes.html
# cd ~
# sudo mkfs -t ext4 /dev/xvdf
# sudo mount /dev/xvdf data
# mkdir data
# sudo mount /dev/xvdf data
# sudo chown data `whoami`

## Copy data

In [ ]:
# cp -R data0 data

### Create Snapshot 

In [3]:
u.get_region()

'us-east-1'

In [59]:
u.get_name(vol)

'imagenet_blank'

In [110]:
ec2.create_snapshot?

In [51]:
snapshot = ec2.create_snapshot(Description=f'{u.get_name(vol)} snapshot', VolumeId=vol.id,)

In [56]:
snapshot.reload()

In [57]:
snapshot.state

'completed'

In [48]:
u.get_name(vol)

'imagenet_blank'

In [ ]:
# give snapshot a name

In [111]:
u.make_name('hi')

[{'Key': 'Name', 'Value': 'hi'}]

In [53]:
vpc.create_tags(Tags=u.make_name(VPC_NAME))


'us-east-1'

## Replicate snapshot in zone

In [62]:
snapshot.

ec2.Snapshot(id='snap-0d37b903e01bb794a')

In [63]:
def create_tags(name):
  return [{
    'ResourceType': 'volume',
    'Tags': [{
        'Key': 'Name',
        'Value': name
    }]
}]

In [64]:
create_tags('imagenet_0')

[{'ResourceType': 'volume', 'Tags': [{'Key': 'Name', 'Value': 'imagenet_0'}]}]

In [73]:
vol0 = ec2.create_volume(Size=300, VolumeType='io1', TagSpecifications=create_tags('imagenet_0'),
                           AvailabilityZone='us-east-1c', SnapshotId=snapshot.id,
                           Iops=10000)

In [74]:
vol0

ec2.Volume(id='vol-07ed8d0f842113290')

In [69]:
def replicate_imagenet(num_replicas, zone, snapshot):
  for i in range(num_replicas):
    ec2.create_volume(Size=300, VolumeType='io1', TagSpecifications=create_tags('imagenet_%02d'%(i)),
                           AvailabilityZone='us-east-1c', SnapshotId=snapshot.id,
                           Iops=10000)

{'ResponseMetadata': {'RequestId': '70a8063b-cfa7-4d19-9c6a-8b3b9b455ae2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding',
   'date': 'Wed, 01 Aug 2018 16:58:16 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [75]:
snapshot.id

'snap-0d37b903e01bb794a'

In [76]:
ec2.Snapshot(snapshot.id)

ec2.Snapshot(id='snap-0d37b903e01bb794a')

In [77]:
snapshot.volume

ec2.Volume(id='vol-0a1265aa66e50ec14')

## List snapshots

In [85]:
snapshots = list(ec2.snapshots.filter(OwnerIds=['self']))

In [86]:
len(snapshots)

12

In [89]:
for snap in snapshots:
  print(snap.id, snap.description)

snap-02c1f2467776319db Copied for DestinationAmi ami-78dfe807 from SourceAmi ami-6583d71d for SourceSnapshot snap-0221e898577adee42. Task created on 1,531,301,312,003.
snap-07df371431e271731 Created by CreateImage(i-0ca86b25ccfdb2bad) for ami-dd9184a2 from vol-09a0982694adda14e
snap-093b8a259897f9cc2 Copied for DestinationAmi ami-72b7bf0d from SourceAmi ami-70306b08 for SourceSnapshot snap-0d100bf7117fc7e89. Task created on 1,532,748,613,402.
snap-02e518719181ccfa7 Copied for DestinationAmi ami-e6090199 from SourceAmi ami-bc227ac4 for SourceSnapshot snap-0d5a3a544dd02d4ca. Task created on 1,532,700,561,717.
snap-0e7edc7112e923950 Copied for DestinationAmi ami-a9bb9bd6 from SourceAmi ami-53c8822b for SourceSnapshot snap-0f26a36b43deea971. Task created on 1,530,640,163,368.
snap-00cecc84f1adf0af0 Created by CreateImage(i-0ddb9b1d8504b342f) for ami-5b524f21 from vol-00cbc9a74e2ebff28
snap-07f0a9369c4974e2f Created by CreateImage(i-016e2869519aa7eef) for ami-f9d6dc83 from vol-009d3f7a768ce

In [90]:
snap.description

'Copied for DestinationAmi ami-e6090199 from SourceAmi ami-bc227ac4 for SourceSnapshot snap-063133a5a7d71c6fe. Task created on 1,532,700,561,717.'

In [ ]:
snap.modify_attribute

In [ ]:
snapshots = list(ec2.snapshots.all())
for v in volumes:
  if v.attachments:
    machine = u.get_name(ec2.Instance(v.attachments[0]['InstanceId']))
  else:
    machine = '<unattached>'
  print(v.id, u.get_name(v), machine)

# Copy snapshot, make public

In [14]:
vol = u.get_volume_dict()['imagenet_blank']

In [ ]:
vol0 = ec2.create_volume(Size=300, VolumeType='io1', TagSpecifications=create_tags('imagenet_0'),
                           AvailabilityZone='us-east-1c', SnapshotId=snapshot.id,
                           Iops=10000)

In [20]:
u.make_name('asdf')

[{'Key': 'Name', 'Value': 'asdf'}]

In [54]:
snapshot = ec2.create_snapshot(Description=f'imagenet18', VolumeId=vol.id,
                    TagSpecifications=[{'ResourceType':'snapshot', 'Tags':u.make_name('imagenet18')}])

In [57]:
# look up by name
list(ec2.snapshots.filter(Filters=[{'Name':'tag:Name', 'Values':['imagenet18']}]))

[ec2.Snapshot(id='snap-08688d235aae5155e')]

In [66]:
# lookup by description
list(ec2.snapshots.filter(Filters=[{'Name':'description', 'Values':['imagenet18']},{'Name':'owner-id', 'Values':['316880547378']}]))

[ec2.Snapshot(id='snap-08688d235aae5155e')]

In [64]:
ec2.snapshots.filter?

In [67]:
snapshot = u.get_snapshot_dict()['imagenet18']

In [59]:
response = snapshot.modify_attribute(
    Attribute='createVolumePermission',
    CreateVolumePermission={
        'Add': [
            {
                'Group': 'all',
            },
        ],
    },
    GroupNames=[
        'string',
    ],
    OperationType='add',
)

In [68]:
snapshot.state

'completed'

# List volumes

In [108]:
  v = list(ec2.volumes.filter(Filters=[{'Name':'tag:Name', 'Values':['imagenet_00']}, {"Name":"availability-zone", 'Values':['us-east-1c']}]).all())


In [109]:
v

[ec2.Volume(id='vol-082fde9b0b4d2c9b1')]

In [107]:
ec2.volumes.filter?

In [ ]:
 v = list(ec2.volumes.filter(Filters=[{'Name':'tag:Name', 'Values':['imagenet_00']}]).all())
